In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import feather
import datetime
from matplotlib_venn import venn2
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgbm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
pd.set_option('display.max_Columns', 100)

In [2]:
cartlog = pd.read_feather('../inputs/cartlog.f')
product_master = pd.read_feather('../inputs/product_master.f')
meta = pd.read_feather('../inputs/meta.f')
user_master = pd.read_feather('../inputs/user_master.f')
test = pd.read_csv('../inputs/test.csv')

product_master['JAN'] = product_master['JAN'].astype(str)

In [3]:
test_sessions = test["session_id"].unique()
print(len(test_sessions))
test_input_log = cartlog[cartlog["session_id"].isin(test_sessions)]

56486


In [4]:
class RetailDataset:
    def __init__(self, thres_sec, cartlog, product, meta):
        self.thres_sec = thres_sec
        self.cartlog = cartlog
        self.product_master = product
        self.meta = meta.copy()
        
        meta['time_elapsed_sec'] = meta['time_elapsed'] * 60
        self.meta.loc[self.meta['time_elapsed_sec'].isnull(), 'time_elapsed_sec'] = thres_sec
        self.target_category_ids = [
            38,  # アイスクリーム__ノベルティー
            110,  # スナック・キャンディー__ガム
            113,  # スナック・キャンディー__シリアル
            114,  # スナック・キャンディー__スナック
            134,  # チョコ・ビスクラ__チョコレート
            171,  # ビール系__RTD
            172,  # ビール系__ノンアルコール
            173,  # ビール系__ビール系
            376,  # 和菓子__米菓
            435,  # 大型PET__無糖茶（大型PET）
            467,  # 小型PET__コーヒー（小型PET）
            537,  # 水・炭酸水__大型PET（炭酸水）
            539,  # 水・炭酸水__小型PET（炭酸水）
            629,  # 缶飲料__コーヒー（缶）
            768,  # 麺類__カップ麺
        ]

    
    def get_log_first_half(self) -> pd.DataFrame:
        """以下の条件を満たすログを取得する
        - 学習期間(2020-08-01の前日まで)のセッションである
        """
        first_half_sessions = set(
            self.meta.query("date < '2020-08-01'")["session_id"].unique()
        )
        return self.cartlog[self.cartlog["session_id"].isin(first_half_sessions)]

    def get_train_output_log(self) -> pd.DataFrame:
        """以下の条件を満たすログを取得する
        - 学習期間(2020-08-01の前日まで)のセッションである
        - 指定した時間(thres_sec)以降にログが存在している
        """
        return pd.merge(
            self.get_log_first_half(),
            self.meta[["session_id", "time_elapsed_sec"]],
            on=["session_id"],
            how="inner",
        ).query("spend_time > time_elapsed_sec")

    def get_train_sessions(self) -> set:
        """以下の条件を満たすセッションを取得する
        - 学習期間(2020-08-01の前日まで)のセッションである
        - 指定した時間(thres_sec)以降にログが存在している
        """
        return set(self.get_train_output_log()["session_id"].unique())

    def get_train_input_log(self) -> pd.DataFrame:
        """以下の条件を満たすログを取得する
        - 学習期間(2020-08-01の前日まで)のセッションである
        - 指定した時間(thres_sec)以降にログが存在している
        - 指定した時間(thres_sec)より前のログである
        """
        train_sessions = self.get_train_sessions()
        return pd.merge(
            self.get_log_first_half()[
                self.get_log_first_half()["session_id"].isin(train_sessions)
            ],
            self.meta[["session_id", "time_elapsed_sec"]],
            on=["session_id"],
            how="inner",
        ).query("spend_time <= time_elapsed_sec")

    def get_payment_sessions(self) -> set:
        """以下の条件を満たすセッションを取得する
        - 決済を行った
        """
        return set(self.cartlog.query("is_payment == 1")["session_id"].unique())

    def agg_payment(self, cartlog) -> pd.DataFrame:
        """セッションごと・商品ごとの購買個数を集計する"""
        # 購買情報は商品のものだけ.
        target_index = cartlog["kind_1"] == "商品"

        # JANコード (vale_1)ごとに商品の購入個数(n_items)を足し算
        agg = (
            cartlog.loc[target_index]
            .groupby(["session_id", "value_1"])["n_items"]
            .sum()
            .reset_index()
        )
        agg = agg.rename(columns={"value_1": "JAN"})
#         agg = agg.astype({"JAN": int})
        return agg

    def get_train_target(self) -> pd.DataFrame:
        """学習で使用するセッションの目的変数を取得する"""
        # 空のターゲット用データフレームを用意する
        train_sessions = self.get_train_sessions()
        train_target = pd.DataFrame(
            np.zeros((len(train_sessions), len(self.target_category_ids))),
            index=train_sessions,
            columns=self.target_category_ids,
        ).astype(int)
        train_target.index.name = "session_id"

        # 集計する
        train_output_log = self.get_train_output_log()
        train_items_per_session_jan = self.agg_payment(train_output_log)
        train_items_per_session_target_jan = pd.merge(
            train_items_per_session_jan,
            self.product_master[["JAN", "category_id"]],
            on="JAN",
            how="inner",
        ).query("category_id in @self.target_category_ids")
        train_target_pos = (
            train_items_per_session_target_jan.groupby(["session_id", "category_id"])[
                "n_items"
            ]
            .sum()
            .unstack()
            .fillna(0)
            .astype(int)
        )
        train_target_pos[train_target_pos > 0] = 1
        train_target_pos[train_target_pos <= 0] = 0

        train_target.loc[train_target_pos.index] = train_target_pos.values
        return train_target[self.target_category_ids].reset_index()

In [5]:
def get_train_log(elapsed_min):
    dataset = RetailDataset(elapsed_min*60, cartlog, product_master, meta)
    train_input_log = dataset.get_train_input_log()
    y_train = dataset.get_train_target()
    print('train_session', y_train.shape)
    return train_input_log, y_train

In [6]:
ELAPSED_MIN = [0, 3, 5, 10]

In [7]:
def save_train_log():
    for elap_min in ELAPSED_MIN:
        train_log, train_y = get_train_log(elap_min)
        train_log = train_log.reset_index(drop=True)
        train_y = train_y.reset_index(drop=True)
        train_y.columns = [str(c) for c in train_y.columns]
        train_log.to_feather('../inputs/train_log_{}.f'.format(elap_min))
        train_y.to_feather('../inputs/train_y_{}.f'.format(elap_min))

In [8]:
def load_train_log():
    train_log = {}
    train_y = {}
    for elap_min in ELAPSED_MIN:
        log = pd.read_feather('../inputs/train_log_{}.f'.format(elap_min))
        y = pd.read_feather('../inputs/train_y_{}.f'.format(elap_min))
        train_log[elap_min] = log
        train_y[elap_min] = y
    return train_log, train_y

In [9]:
# save_train_log()

### ユーザ情報を取得する

In [10]:
user_features = pd.merge(
    meta[["session_id", "user_id"]],
    user_master,
    on="user_id",
    how="left",
).drop(columns=["user_id"])
user_features.head()

,session_id,age,gender
0,0,40.0,1.0
1,1,30.0,0.0
2,2,50.0,1.0
3,3,30.0,0.0
4,4,30.0,1.0


### セッション単位で集計

In [11]:
def get_session_features(input_log):
    n_actions = input_log.groupby(["session_id"]).size().rename("n_actions")
    n_add_items = input_log[input_log['kind_1'] == '商品'].groupby(["session_id"]).size().rename("n_add_items")
    mean_spend_time = input_log.groupby(["session_id"])["spend_time"].mean()

    session_features = pd.concat([
        n_actions,
        n_add_items,
        mean_spend_time,
    ], axis=1)
    return session_features

### 特徴量を集約する

In [12]:
def get_features(features, session):
    out = pd.DataFrame({"session_id": session})
    out = pd.merge(out, features, on="session_id", how="left")
    out = pd.merge(out, user_features, on="session_id", how="left")
    assert len(session) == len(out)
    return out

In [13]:
train_log_list, train_y_list = load_train_log()

In [14]:
def get_train_features(elapsed_min):
#     dataset = RetailDataset(elapsed_min*60, cartlog, product_master, meta)
#     train_input_log = dataset.get_train_input_log()
#     y_train = dataset.get_train_target()
    train_input_log = train_log_list[elapsed_min]
    y_train = train_y_list[elapsed_min]
    print('train_session', y_train.shape)
    
    train_session_features = get_session_features(train_input_log)
    train_features = get_features(train_session_features, y_train['session_id'])
    
    return train_features, y_train

In [15]:
def get_test_feature(elapsed_min):
    test_meta = meta[meta['session_id'].isin(test_sessions)]
    test_meta = test_meta[test_meta['time_elapsed'] == elapsed_min]
    test_input_elapsed = pd.merge(test_input_log, test_meta[['session_id']], on='session_id', how='left')
    print(len(test_input_elapsed))
    
    test_session_features = get_session_features(test_input_elapsed)
    test_features = get_features(test_session_features, test_meta['session_id'])
    print('test_session', len(test_meta))
    return test_features

In [16]:
n_fold = 4

In [17]:
lgbm_param = {
    'objective' : 'binary',
    'boosting_type': 'gbdt',
    'metric': 'auc',
    'seed' : 0,
    'learning_rate':  0.1,
    'verbose': -1
}

In [18]:
def train_lgbm(X, y, params=lgbm_param):

    fold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=0)
    cv = fold.split(X, y)
    
    models = []
    oof_pred = np.zeros_like(y, dtype=np.float)

    for i, (idx_train, idx_valid) in enumerate(cv): 
        x_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
        x_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
#         print('x_train', x_train.shape)
#         print('x_valid', x_valid.shape)
        
        lgbm_train = lgbm.Dataset(x_train, y_train)
        lgbm_eval = lgbm.Dataset(x_valid, y_valid, reference=lgbm_train)
        
        lgbm_model = lgbm.train(params, 
                                                    lgbm_train, 
                                                    valid_sets=lgbm_eval,
                                                    num_boost_round=10000,
                                                    early_stopping_rounds=100,
                                                    verbose_eval=-1)
        y_pred = lgbm_model.predict(x_valid, num_iteration=lgbm_model.best_iteration)
        
        oof_pred[idx_valid] = y_pred
        models.append(lgbm_model)
#         print('--- Fold {} Score: {:.4f} ---'.format(i, roc_auc_score(y_valid, y_pred)))

    score = roc_auc_score(y, oof_pred)
    print('--- FINISHED \ whole score: {:.4f} ---'.format(score))
    return oof_pred, models, score

In [19]:
def predict(models, feature):
    pred_list = []
    for i, model in enumerate(models):
        pred = model.predict(feature, num_iteration = model.best_iteration)
        pred_list.append(pred)
    
    score = np.mean(pred_list, axis=0)
    return score

In [27]:
%%time
df_pred_all = pd.DataFrame()
for elapsed_min in ELAPSED_MIN:
    print(f'===== {elapsed_min} =====')
    train_features, y_train = get_train_features(elapsed_min)
    test_features = get_test_feature(elapsed_min)

    df_pred = pd.DataFrame(index=test_features['session_id'])
    models_list = []
    for target in y_train.columns:
        if target == 'session_id':
            continue
        
        print(f"---- id = {target} -----")
        oof, models, score = train_lgbm(train_features, y_train[target])
        models_list.append(models)

        pred = predict(models, test_features)
        df_pred[target] = pred
    
    df_pred_all = pd.concat([df_pred_all, df_pred])
    print(len(df_pred_all))

===== 0 =====
train_session (641293, 16)
333059
test_session 14277
---- id = 38 -----
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[70]	valid_0's auc: 0.606096
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[39]	valid_0's auc: 0.608156
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[73]	valid_0's auc: 0.605691
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[92]	valid_0's auc: 0.605563
--- FINISHED \ whole score: 0.6063 ---
---- id = 110 -----
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[28]	valid_0's auc: 0.556039
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[129]	valid_0's auc: 0.547201
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[65]	valid_0's auc: 0.57322
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[89]	valid_0's auc: 0.576056
--- FINISHED \ whole score: 0.5741 ---
14277
===== 3 =====
train_session (509646, 16)
333059
test_session 11304
---- id = 38 -----
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[36]	valid_0's auc: 0.604489
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[36]	valid_0's auc: 0.611978
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[28]	valid_0's auc: 0.60705
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[56]	valid_0's auc: 0.603103
--- FINISHED \ whole score: 0.6065 ---
---- id = 110 -----
Training until validation scores don't improve for 100 r

--- FINISHED \ whole score: 0.5821 ---
---- id = 768 -----
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[106]	valid_0's auc: 0.558798
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[46]	valid_0's auc: 0.563976
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[44]	valid_0's auc: 0.561621
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[36]	valid_0's auc: 0.562355
--- FINISHED \ whole score: 0.5615 ---
25581
===== 5 =====
train_session (481755, 16)
333059
test_session 14072
---- id = 38 -----
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[57]	valid_0's auc: 0.604629
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[57]	valid_0's auc: 0.597007
Training until validation scores don't improve for 10

Early stopping, best iteration is:
[35]	valid_0's auc: 0.578838
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[30]	valid_0's auc: 0.583426
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[18]	valid_0's auc: 0.585385
--- FINISHED \ whole score: 0.5792 ---
---- id = 768 -----
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[77]	valid_0's auc: 0.557189
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[106]	valid_0's auc: 0.555849
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[32]	valid_0's auc: 0.558653
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[51]	valid_0's auc: 0.559346
--- FINISHED \ whole score: 0.5575 ---
39653
===== 10 =====
train_session (404500, 16)
333059
test_session 16833
---- id

Early stopping, best iteration is:
[25]	valid_0's auc: 0.573397
--- FINISHED \ whole score: 0.5722 ---
---- id = 629 -----
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[98]	valid_0's auc: 0.579131
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[9]	valid_0's auc: 0.603735
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[24]	valid_0's auc: 0.589251
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[25]	valid_0's auc: 0.58032
--- FINISHED \ whole score: 0.5828 ---
---- id = 768 -----
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[56]	valid_0's auc: 0.561492
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[34]	valid_0's auc: 0.564053
Training until validation scores don't improve for 100 rounds
Ea

In [38]:
df_pred_all.head()

,38,110,113,114,134,171,172,173,376,435,467,537,539,629,768
session_id,,,,,,,,,,,,,,,
663721,0.119591,0.009427,0.042495,0.177740,0.099918,0.092893,0.021715,0.118988,0.078520,0.074855,0.027024,0.039832,0.055056,0.010882,0.085923
663761,0.133678,0.009714,0.052441,0.254966,0.141103,0.103017,0.020711,0.110578,0.077371,0.053650,0.032172,0.040798,0.046811,0.009557,0.086664
663763,0.091964,0.008202,0.027734,0.220496,0.082287,0.105156,0.023026,0.111243,0.056153,0.083343,0.037564,0.039376,0.052735,0.012231,0.098819
663775,0.119591,0.009427,0.042495,0.177740,0.099918,0.092893,0.021715,0.118988,0.078520,0.074855,0.027024,0.039832,0.055056,0.010882,0.085923
663778,0.137751,0.010739,0.026849,0.167242,0.073931,0.062505,0.014710,0.103499,0.075016,0.063450,0.028383,0.048642,0.073698,0.013243,0.124506


In [39]:
assert len(df_pred_all) == len(test)

In [40]:
submission = pd.merge(test[['session_id']], df_pred_all.reset_index(), on='session_id', how='inner')
assert len(submission) == len(test)

In [41]:
submission.drop(columns='session_id').to_csv('../outputs/submission.csv', index=False)